In [19]:
from IPython.utils import io
import pandas as pd
import os
from pathlib import Path
from matplotlib import pyplot as plt
from scipy.signal import find_peaks
import numpy as np
import seaborn as sns
import time
import joblib
from os.path import exists
import shutil
import sys
import time

from sklearn.model_selection import train_test_split

#From my EEG package 
import run_expts
import constants

#Let me see as many results as I want to see
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
classifier_list  = [{'C_I_F': CanonicalIntervalForest()}, {'Dr_C_I_F': DrCIF()}]   

NameError: name 'CanonicalIntervalForest' is not defined

In [ ]:
# expt_1_N3_C_I_F_int_df.h5
# expt_1_N3_Dr_C_I_F_raw_clfs_results.h5

In [20]:
data_types = ['Wake' , 'N1' , 'N2' , 'N3' , 'REM']

expt_num = [1,2,3,4]

clf_list = ['Rocket', 'C_I_F' , 'Dr_C_I_F']

#Results dictionary structure : {model : {expt_type : {folder : results_df }}}
results_dict = {}
for clf in clf_list:
    results_dict[clf] = {1 : {} , 2 : {} , 3: {} , 4 : {}}

files_not_found = []
found_files = []

for num in [1,2,3,4]:
    for datatype in data_types:
        for clf in clf_list:
            try:
                if clf == 'Rocket':
                    save_path = 'Results/Results_3/' + 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
                else:
                    save_path = 'Results/Results_3/' +  'expt_' + str(num) + '_' + datatype + '_' + clf + '_raw_clfs_results.h5'
                    
#                 file_name = 'expt_' + str(num) + '_' + datatype + '_raw_clfs_results.h5'
                
                # os.path.dirname(os.getcwd()) + '/' 
                # This generates path to folder one up from folder in which script is executing

                results_df = pd.read_hdf(os.path.dirname(os.getcwd()) + '/' +save_path , key='df', mode='r')
                found_files.append(save_path)
                
            except FileNotFoundError:
                files_not_found.append(save_path)
                continue
                
            res_df = run_expts.generate_subset_acc_std(results_df, return_df = True)           
            results_dict[clf][num][datatype] = res_df

In [26]:
for clf in ['Rocket', 'C_I_F' , 'Dr_C_I_F']:
    for num in [3,4]:
        print('Expt Type: ' + str(num))
        not_displayed = []
        for key in list(results_dict[clf][num].keys() ):
            df = results_dict[clf][num][key].copy()
            cols_to_drop = [col for col in df.columns if df[col].mean() < 0.5]
            df = df.drop(cols_to_drop, axis = 1 )
            if len(df.columns) < 2:
                not_displayed.append(key)
                continue
            print(key)
            display(df)
        print(not_displayed)

Expt Type: 3
Wake


,random_state,Prefrontal_ROCKET_acc
0,1,0.517359
1,2,0.516667


['N1', 'N2', 'N3', 'REM']
Expt Type: 4
['Wake', 'N1', 'N2', 'N3', 'REM']
Expt Type: 3
[]
Expt Type: 4
[]
Expt Type: 3
[]
Expt Type: 4
[]


## Put the data in a dataframe structure that can easily be plotted

In [23]:
#Defining the dataframe dictionary to be filled up

expt_info = [ '1. PD vs HC', '2. (PD , PD+RBD) vs HC', '3. (PD + PD&RBD) vs HC vs RBD  multiclass', 
             '4. PD vs PD+RBD vs RBD vs HC  multiclass' ]

df_cols = ['Expt_type num', 'Folder', 'Classifier Name', 'Classifier', 'EEG Type',
       'Brain Region', 'Accuracy', 'Accuracy_std', 'TS processing']

df_dict = {}
for col in df_cols:
    df_dict[col] = []

regions = constants.regions

In [27]:
df_dict

{'Expt_type num': [],
 'Folder': [],
 'Classifier Name': [],
 'Classifier': [],
 'EEG Type': [],
 'Brain Region': [],
 'Accuracy': [],
 'Accuracy_std': [],
 'TS processing': []}

In [29]:
proc = 'shorter_expts'
for clf in clf_list:
    #Go through EEG types
    for eeg_type_res in ['Wake' , 'N1' , 'N2' , 'N3' , 'REM']:
               
        for expt_num in [1,2,3,4]:
            #Fill up the results_dict with the appropriate values
            #Grab accuracy and std performance for each region
            try:
                mean_df = pd.DataFrame(results_dict[clf][expt_num][eeg_type_res].mean(axis = 0) ).T
            
            except KeyError:
                continue
            for region in regions:
                #For each region take mean accuracy and std
                region_cols = [col for col in mean_df.columns if region in col]
                acc_col = [col for col in region_cols if 'acc' in col]
                std_col = [col for col in region_cols if 'std' in col]
                region_acc = mean_df[acc_col].values[0][0]
                region_acc_std = mean_df[std_col].values[0][0]
                
                #Now fill up the dataframe
                df_dict['Expt_type num'].append(expt_num)
                df_dict['Folder'].append(eeg_type_res)
                df_dict['Classifier Name'].append(clf)
                df_dict['Classifier'].append(clf)
                df_dict['EEG Type'].append(eeg_type)
                df_dict['Brain Region'].append(region)
                df_dict['Accuracy'].append(region_acc)
                df_dict['Accuracy_std'].append(region_acc_std)
                df_dict['TS processing'].append(proc)

results_df = pd.DataFrame(df_dict)

In [30]:
results_df

,Expt_type num,Folder,Classifier Name,Classifier,EEG Type,Brain Region,Accuracy,Accuracy_std,TS processing
0,1,Wake,Rocket,Rocket,Wake,Prefrontal,0.469048,0.107638,shorter_expts
1,1,Wake,Rocket,Rocket,Wake,Frontal,0.571429,0.147778,shorter_expts
2,1,Wake,Rocket,Rocket,Wake,Left Frontal,0.588095,0.220717,shorter_expts
3,1,Wake,Rocket,Rocket,Wake,Right Frontal,0.435714,0.153163,shorter_expts
4,1,Wake,Rocket,Rocket,Wake,Central,0.571429,0.183035,shorter_expts
5,1,Wake,Rocket,Rocket,Wake,Left Central,0.423810,0.118051,shorter_expts
6,1,Wake,Rocket,Rocket,Wake,Right Central,0.490476,0.156206,shorter_expts
7,1,Wake,Rocket,Rocket,Wake,Left Temporal,0.488095,0.156141,shorter_expts
8,1,Wake,Rocket,Rocket,Wake,Right Temporal,0.421429,0.134854,shorter_expts
9,1,Wake,Rocket,Rocket,Wake,Parietal,0.504762,0.172149,shorter_expts


## This portion of the notebook plots accuracy by region (to view trends)

In [31]:
results_df_sorted = results_df.sort_values(by = ['Accuracy'], ascending = False)
results_df_sorted.head(12)

,Expt_type num,Folder,Classifier Name,Classifier,EEG Type,Brain Region,Accuracy,Accuracy_std,TS processing
276,2,Wake,C_I_F,C_I_F,Wake,Right Frontal,0.747421,0.113890,shorter_expts
24,2,Wake,Rocket,Rocket,Wake,Right Parietal,0.703175,0.099572,shorter_expts
394,1,N1,Dr_C_I_F,Dr_C_I_F,Wake,Central,0.679553,0.149004,shorter_expts
25,2,Wake,Rocket,Rocket,Wake,Occipital,0.670238,0.092990,shorter_expts
273,2,Wake,C_I_F,C_I_F,Wake,Prefrontal,0.668452,0.103202,shorter_expts
59,1,N1,Rocket,Rocket,Wake,Left Temporal,0.666176,0.124510,shorter_expts
57,1,N1,Rocket,Rocket,Wake,Left Central,0.664358,0.120883,shorter_expts
23,2,Wake,Rocket,Rocket,Wake,Left Parietal,0.645635,0.142529,shorter_expts
269,1,Wake,C_I_F,C_I_F,Wake,Parietal,0.635714,0.206694,shorter_expts
275,2,Wake,C_I_F,C_I_F,Wake,Left Frontal,0.632341,0.147263,shorter_expts
